# Bank account management

The goal of this lab is to implement code that manages bank accounts.  
You will implement this code twice: first in a purely imperative style (using dictionaries and functions), and then a second time where each bank account is represented by an object.

## Specifications
Here are the specifications that your implementations must satisfy:

 - Four types of accounts are possible:
    - current account
    - deposit account
    - Livret A (regulated savings account)
    - joint account
 - An account is made up of an owner and a balance.
 - An owner can deposit or withdraw money (except for some special cases; see below).
 - An owner can transfer money (with the same constraints as for withdrawals and deposits) from one of their accounts to another of their accounts or to another owner's account.

Depending on the type of account, a number of constraints must be checked:
 - A deposit account may not have a negative balance.
 - A Livret A may not have a balance greater than €29,500.
 - A joint account must have two owners.

## Sequence of operations

The following code defines the function `run_actions` which will perform a sequence of banking operations and a few checks at the end.
For this function to work, it needs five functions to be implemented.

Start by reading and understanding the code below.

In [ ]:
ACCOUNT_TYPES = {"CURRENT_ACCOUNT", "LIVRET_A", "DEPOSIT_ACCOUNT", "JOINT_ACCOUNT"}


def run_actions() -> None:
    # Isabelle opens a current account and deposits €1300
    current_isabelle = create_account("CURRENT_ACCOUNT", "isabelle")
    deposit(current_isabelle, "isabelle", 1300)

    # Jean opens a deposit account and deposits €1000
    deposit_jean = create_account("DEPOSIT_ACCOUNT", "jean")
    deposit(deposit_jean, "jean", 1000)

    # Jean opens a Livret A and deposits €1200
    livret_a_jean = create_account("LIVRET_A", "jean")
    deposit(livret_a_jean, "jean", 1200)

    # Isabelle and Jean open a joint account and deposit €6500
    joint_isabelle = create_account("JOINT_ACCOUNT", "isabelle", "jean")
    deposit(joint_isabelle, "isabelle", 6500)

    # Isabelle transfers €500 from her current account to the joint account
    transfer(current_isabelle, joint_isabelle, "isabelle", 500)

    # Jean withdraws €60 from his Livret A
    withdraw(livret_a_jean, "jean", 60)

    # START OF TESTS
    # Test the balances of the accounts
    assert get_balance(current_isabelle, "isabelle") == 800
    assert get_balance(livret_a_jean, "jean") == 1140
    assert get_balance(joint_isabelle, "isabelle") == 7000

    # The account may not have a negative balance
    try:
        withdraw(deposit_jean, "jean", 2000)
    except ValueError:
        print("Insufficient balance correctly detected")
    else:
        print("An error should have occurred")

    # The Livret A may not have more than €29,500
    try:
        deposit(livret_a_jean, "jean", 29_000)
    except ValueError:
        print("Maximum threshold detected")
    else:
        print("An error should have occurred")

## Imperative implementation

The `run_actions` function must be able to execute normally. You must implement the following functions:

 - `create_account`
 - `deposit`
 - `withdraw`
 - `transfer`
 - `get_balance`

It is up to you to implement these functions. You may add any helper functions or data structures you like, but do **not** create classes for now.

Once this code has been completed, the function `run_actions` should run successfully.

In [ ]:
from typing import Any


def create_account(
    account_type: str, owner: str, owner2: str | None = None
) -> dict[str, Any]:
    # Your code here
    pass


def deposit(account: dict[str, Any], requester: str, amount: int) -> None:
    # Your code here
    pass


def withdraw(account: dict[str, Any], requester: str, amount: int) -> None:
    # Your code here
    pass


def transfer(
    source_account: dict[str, Any],
    target_account: dict[str, Any],
    requester: str,
    amount: int,
) -> None:
    # Your code here
    pass


def get_balance(account: dict[str, Any], requester: str) -> int:
    # Your code here
    pass

In [ ]:
# run_actions()  # Run this once you have implemented the code above

### Solution

In [ ]:
from typing import Any


def create_account(
    account_type: str, owner: str, owner2: str | None = None
) -> dict[str, Any]:
    assert (account_type == "JOINT_ACCOUNT") != (owner2 is None)
    return {
        "type": account_type,
        "balance": 0,
        "owner": owner,
        "owner2": owner2,
    }


def deposit(account: dict[str, Any], requester: str, amount: int) -> None:
    _check_owner(account, requester)
    _check_constraints(account, amount)
    account["balance"] += amount


def withdraw(account: dict[str, Any], requester: str, amount: int) -> None:
    _check_owner(account, requester)
    _check_constraints(account, -amount)
    account["balance"] -= amount


def transfer(
    source_account: dict[str, Any],
    target_account: dict[str, Any],
    requester: str,
    amount: int,
) -> None:
    _check_owner(source_account, requester)
    _check_constraints(source_account, -amount)
    _check_constraints(target_account, amount)
    source_account["balance"] -= amount
    target_account["balance"] += amount


def get_balance(account: dict[str, Any], requester: str) -> int:
    _check_owner(account, requester)
    return account["balance"]


def _check_owner(account: dict[str, Any], requester: str) -> None:
    if not _is_owner(account, requester):
        raise ValueError(
            f"{requester} is not allowed to operate on this account."
        )


def _check_constraints(account: dict[str, Any], amount: int) -> None:
    if account["type"] == "LIVRET_A" and account["balance"] + amount > 29_500:
        raise ValueError("Maximum threshold of €29,500 reached.")
    if (
        account["type"] == "DEPOSIT_ACCOUNT"
        and account["balance"] + amount < 0
    ):
        raise ValueError("Insufficient balance.")


def _is_owner(account: dict[str, Any], person: str) -> bool:
    return person in {account["owner"], account["owner2"]}

Another imperative, non-OOP implementation, using a dataclass:

In [ ]:
from dataclasses import dataclass


@dataclass
class Account:
    type: str
    balance: int
    owner: str
    owner2: str | None

def create_account(
    account_type: str, owner: str, owner2: str | None = None
) -> Account:
    assert (account_type == "JOINT_ACCOUNT") != (owner2 is None)
    return Account(type=account_type, balance=0, owner=owner, owner2=owner2)


def deposit(account: Account, requester: str, amount: int) -> None:
    _check_owner(account, requester)
    _check_constraints(account, amount)
    account.balance += amount


def withdraw(account: Account, requester: str, amount: int) -> None:
    _check_owner(account, requester)
    _check_constraints(account, -amount)
    account.balance -= amount


def transfer(
    source_account: Account,
    target_account: Account,
    requester: str,
    amount: int,
) -> None:
    _check_owner(source_account, requester)
    _check_constraints(source_account, -amount)
    _check_constraints(target_account, amount)
    source_account.balance -= amount
    target_account.balance += amount


def get_balance(account: Account, requester: str) -> int:
    _check_owner(account, requester)
    return account.balance


def _check_owner(account: Account, requester: str) -> None:
    if not _is_owner(account, requester):
        raise ValueError(
            f"{requester} is not allowed to operate on this account."
        )


def _check_constraints(account: Account, amount: int) -> None:
    if account.type == "LIVRET_A" and account.balance + amount > 29_500:
        raise ValueError("Maximum threshold of €29,500 reached.")
    if (
        account.type == "DEPOSIT_ACCOUNT"
        and account.balance + amount < 0
    ):
        raise ValueError("Insufficient balance.")


def _is_owner(account: Account, person: str) -> bool:
    return person in {account.owner, account.owner2}

In [ ]:
run_actions()

## Object-oriented implementation

Same exercise as before, but this time you must implement the different kinds of bank accounts as objects.

All account classes that you create must inherit from the `Account` class, which you must also define.

In [ ]:
def run_actions() -> None:
    # Isabelle opens a current account and deposits €1300
    current_isabelle = CurrentAccount("isabelle")
    current_isabelle.deposit("isabelle", 1300)

    # Jean opens a deposit account and deposits €1000
    deposit_jean = DepositAccount("jean")
    deposit_jean.deposit("jean", 1000)

    # Jean opens a Livret A and deposits €1200
    livret_a_jean = LivretA("jean")
    livret_a_jean.deposit("jean", 1200)

    # Isabelle and Jean open a joint account and deposit €6500
    joint_isabelle = JointAccount("isabelle", "jean")
    joint_isabelle.deposit("isabelle", 6500)

    # Isabelle transfers €500 from her current account to the joint account
    current_isabelle.transfer("isabelle", joint_isabelle, 500)

    # Jean withdraws €60 from his Livret A
    livret_a_jean.withdraw("jean", 60)

    # START OF TESTS
    # Test the balances of the accounts
    assert current_isabelle.balance("isabelle") == 800
    assert livret_a_jean.balance("jean") == 1140
    assert joint_isabelle.balance("isabelle") == 7000

    # The account may not have a negative balance
    try:
        deposit_jean.withdraw("jean", 2000)
    except ValueError:
        print("Insufficient balance correctly detected")
    else:
        print("An error should have occurred")

    # The Livret A may not have more than €29,500
    try:
        livret_a_jean.deposit("jean", 29_000)
    except ValueError:
        print("Maximum threshold detected")
    else:
        print("An error should have occurred")

In [ ]:
class Account:
    def __init__(self, owner: str):
        self._owner = owner
        self._balance = 0

    def balance(self, requester: str) -> int:
        # Your code here
        raise NotImplementedError

    def deposit(self, requester: str, amount: int) -> None:
        # Your code here
        raise NotImplementedError

    def withdraw(self, requester: str, amount: int) -> None:
        # Your code here
        raise NotImplementedError

    def transfer(self, requester: str, other: "Account", amount: int) -> None:
        # Your code here
        raise NotImplementedError


class CurrentAccount(Account):
    # Your code here
    pass


class DepositAccount(Account):
    # Your code here
    pass


class LivretA(Account):
    # Your code here
    pass


class JointAccount(Account):
    # Your code here
    pass

In [ ]:
# run_actions()  # Run this once you have implemented the code above

### Solution

In [ ]:
class Account:
    def __init__(self, owner: str) -> None:
        self._owner = owner
        self._balance = 0

    def _check_owner(self, requester: str) -> None:
        if not self.is_owner(requester):
            raise ValueError(
                f"{requester} is not allowed to operate on this account."
            )

    def _check_constraints(self, amount: int) -> None:
        pass

    def deposit(self, requester: str, amount: int) -> None:
        self._check_owner(requester)
        self._check_constraints(amount)
        self._balance += amount

    def withdraw(self, requester: str, amount: int) -> None:
        self._check_owner(requester)
        self._check_constraints(-amount)
        self._balance -= amount

    def balance(self, requester: str) -> int:
        self._check_owner(requester)
        return self._balance

    def transfer(self, requester: str, other: "Account", amount: int) -> None:
        self._check_owner(requester)
        self._check_constraints(-amount)
        other._check_constraints(amount)
        self._balance -= amount
        other._balance += amount

    def is_owner(self, owner: str) -> bool:
        return self._owner == owner


class CurrentAccount(Account):
    pass


class DepositAccount(Account):
    def _check_constraints(self, amount: int) -> None:
        if self._balance + amount < 0:
            raise ValueError("No overdraft allowed.")


class LivretA(Account):
    def _check_constraints(self, amount: int) -> None:
        if self._balance + amount > 29_500:
            raise ValueError("Limit exceeded.")


class JointAccount(Account):
    def __init__(self, owner: str, owner2: str) -> None:
        super().__init__(owner)
        self._owner2 = owner2

    def is_owner(self, owner: str) -> bool:
        return owner in {self._owner, self._owner2}

In [ ]:
run_actions()